In [1]:
print("Importa librerie", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import keras_tuner as kt
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
import pickle
print(".", end="", flush=True)
from sklearn.model_selection import train_test_split
print(".", end="", flush=True)
from kerastuner.engine.hypermodel import HyperModel
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)
print("ok")

Importa librerie

2023-10-30 18:07:46.888715: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 18:07:46.888774: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 18:07:46.888856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 18:07:46.899285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 18:07:47.996780: W tensorflow/compiler/

.Using TensorFlow backend
....

ImportError: cannot import name 'Bidirectional' from 'tensorflow.python.keras.layers' (/home/enrico/Documenti/Borsa/.venv/lib/python3.10/site-packages/tensorflow/python/keras/layers/__init__.py)

In [ ]:
print("Versione TensorFlow:", tf.__version__)
print("Versione CUDA:", tf.sysconfig.get_build_info()['cuda_version'])
print("Versione cuDNN:", tf.sysconfig.get_build_info()['cudnn_version'])
import psutil

def print_memory_usage():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_visible_devices(gpu, 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        

In [ ]:
n_simboli_addestramento = 500
epochs=50
batch_size=32
perc_dati = "dati"
set_file_x_y = f"_{n_simboli_addestramento}"
initial_lr = 0.001

if not os.path.exists('dati'):
    os.makedirs('dati')

if os.path.exists("lista_ticker.parquet"):
    print("Caricamento lista_ticker esistente", flush=True)
    lista_ticker = pd.read_parquet("lista_ticker.parquet")
else:
    print("Download lista ticker", flush=True)
    lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
    lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
    lista_ticker = lista_ticker.loc[(lista_ticker["Categoria"] != "D"), :]
    lista_ticker.to_parquet("lista_ticker.parquet")

if n_simboli_addestramento == 'Tutti': 
    n_simboli_addestramento = len(lista_ticker)

# Impostazioni
objective = kt.Objective("val_f1", direction="max")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1', factor=0.5, patience=5, min_lr=0.0001, mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1', patience=10, restore_best_weights=True, mode='max')
callbacks = [early_stopping, reduce_lr]

X, Y = fx.set_di_tuning(lista_ticker)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print_memory_usage()
input_shape = (fx.n_timesteps, fx.n_features)
hypermodel = MixedHyperModel(fx.n_timesteps, fx.n_features)

In [ ]:
print("Tuning")

print_memory_usage()
model, par, bayesian_search = bay(hypermodel, objective, X_train, Y_train, X_val, Y_val)
print_memory_usage()

trials = bayesian_search.oracle.trials

results = []

for trial in trials.values():
    trial_metrics = trial['metrics']
    val_f1 = trial_metrics['val_f1']['observations'][0]['value'] 
    results.append({
        'trial_id': trial['trial_id'],
        'val_f1': val_f1,  
        'hyperparameters': trial['hyperparameters'].values
    })

results_df = pd.DataFrame(results)
results_df.to_excel('results_df.xlsx')
    
model.save("tuning")
with open('hyperparameters.pkl', 'wb') as f:
    pickle.dump(par, f)

#addestramento(lista_ticker)
#    
print("ok")




In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
